# Generate list of root users

In This script, I start with one root Steam user (Smiley), and using network search generate a larger list of users. The basic idea, is that by starting with one user that has at least some friends, we can look at this users friends, and then the friends of the users friends, and so on. Because of the architecture of social networks, we should be able to find nearly every Steam user using this network search approach after several iterations (provided they have friends, that is). 

In [1]:
import requests
import json
from pandas.io.json import json_normalize
import pandas as pd
import numpy as np

# Query Steam's API to load data for a certain user
steamID = '76561197960434622' # smiley

# Below we query a list of the Steam user ID's for smiley's friends
key     = 'Insert your key here'
method  = 'GetFriendList'
currentUser = requests.get("http://api.steampowered.com/ISteamUser/{}/v0001/?key={}&steamid={}&relationship=friend".format(method,key,steamID))
json2pyData = json.loads(currentUser.content)
rootFriends = pd.DataFrame(json2pyData['friendslist']['friends'])

## Make a list containing the friends of Smiley's friends

Here we loop through each of Smiley's friends, and then query a list of their friends. So the people in this list are second degree friends of Smiley. 

In [4]:
allIDs = pd.DataFrame()
i = 0
for friend in rootFriends['steamid']:
    i = i+1
    method  = 'GetFriendList' # API method to get friends list
    currentUser = requests.get("http://api.steampowered.com/ISteamUser/{}/v0001/?key={}&steamid={}&relationship=friend".format(method,key,friend))
    json2pyData = json.loads(currentUser.content)
    try: 
        friends = pd.DataFrame(json2pyData['friendslist']['friends'])
        tmpIDs  = friends['steamid']
        allIDs = allIDs.append(friends,ignore_index=True)
        print("Processing friend #{} of Smiley".format(i),end = "\r")
    except:
        pass
print("Detected {} second degree friends of Smiley".format(len(allIDs)))

Detected 27257 second degree friends of Smiley


As you can see in the cell below, just this simple network search based off of one root user (Smiley in this case) yielded a list of more than 26 thousand unique Steam user ID's. 

In [5]:
allIDs = allIDs.drop_duplicates()
print("{} unique friends of friends from root user (Smiley)".format(len(allIDs)))
allIDs.to_pickle('alIDs_initial.pkl')

26475 unique friends of friends from root user (Smiley)
